In [1]:
import os
import glob
import pandas as pd
import numpy as np
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cf

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
dir_base = os.path.join('/','media','arturo','Arturo','Data','Brazil','MERGE_INPE')

In [3]:
DATA = xr.open_dataset(os.path.join(dir_base,'MERGE-INPE_SA_1h_2024_01_01_2024_12_31.nc'))

In [4]:
DAILY = DATA['PRE'].resample(time='1D').sum()

In [ ]:
PRE_xr = xr.Dataset(data_vars={"PRE": (("time","lat","lon"), DAILY.data[:,::-1,:])},
                    coords={'time': DAILY.time.data, 'lat': DAILY.lat.data, 'lon': DAILY.lon.data},
                    attrs=dict(description=f'Daily MERGE-NOW INPE precipitation forin SA',))

PRE_xr.PRE.attrs["units"] = "mm/h"
PRE_xr.PRE.attrs["long_name"] = "precipitation"
PRE_xr.PRE.attrs["origname"] = "precipitation"

PRE_xr.lat.attrs["units"] = "degrees_north"
PRE_xr.lat.attrs["long_name"] = "Latitude"

PRE_xr.lon.attrs["units"] = "degrees_east"
PRE_xr.lon.attrs["long_name"] = "Longitude"

PRE_out = os.path.join(dir_base,f'MERGE-NOW-INPE_SA_1dy_2024.nc')
print(f'Export PRE data to {PRE_out}')
PRE_xr.to_netcdf(PRE_out)

Export PRE data to /media/arturo/Arturo/Data/Brazil/MERGE_INPE/MERGE-NOW_SA_1dy_2024.nc


In [5]:
lon2d, lat2d = np.meshgrid(DATA.lon, DATA.lat)

In [ ]:
levels = np.arange(0,36,2)
# levels = None

cmap = plt.cm.Spectral_r

proj = ccrs.PlateCarree()
fig = plt.figure(figsize=(3,3),dpi=300)
gs = gridspec.GridSpec(1,1)

#==================================================================================================
ax1 = plt.subplot(gs[0, 0], projection = proj)
# ax1.set_extent([-87,-30,-50,15],crs=proj)
ax1.add_feature(cf.COASTLINE,linewidth=0.3, linestyle='-',edgecolor='k')
ax1.add_feature(cf.BORDERS, linewidth=0.3, linestyle='-', edgecolor='k')

a1 = ax1.contourf(lon2d, lat2d, np.mean(PRE_xr.PRE.values,axis=0),levels=levels, cmap=cmap)

gl = ax1.gridlines(crs=proj,draw_labels=True,linewidth=0.2,color='gray',alpha=0.5,linestyle='--')
gl.top_labels = False
gl.bottom_labels = True
gl.left_labels = True
gl.right_labels = False
gl.xlabel_style = {'size': 6, 'color': 'k'}
gl.ylabel_style = {'size': 6, 'color': 'k'}
